# Práctica 7: Modelos del lenguaje

## Preprocesamiento

### Leer archivo y generar lista de oraciones

In [1]:
path = './el-quijote.txt'

In [2]:
import ast
encoding = 'utf-8-sig'

try:
    with open(path, 'r', encoding=encoding) as f:
        contents = f.readlines()
        #parsed_corpus = [ ast.literal_eval(x) for x in contents ]
except UnicodeDecodeError:
    print("Error: Unable to decode the file with the specified encoding.")
    
len(contents)

35522

In [3]:
contents[10:20]

['podadera. Frisaba la edad de nuestro hidalgo con los cincuenta años; era de\n',
 'complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo\n',
 'de la caza. Quieren decir que tenía el sobrenombre de Quijada, o Quesada,\n',
 'que en esto hay alguna diferencia en los autores que deste caso escriben;\n',
 'aunque, por conjeturas verosímiles, se deja entender que se llamaba\n',
 'Quejana. Pero esto importa poco a nuestro cuento; basta que en la narración\n',
 'dél no se salga un punto de la verdad.\n',
 '\n',
 'Es, pues, de saber que este sobredicho hidalgo, los ratos que estaba\n',
 'ocioso, que eran los más del año, se daba a leer libros de caballerías, con\n']

In [4]:
from functools import reduce
import operator

def get_sentences(corpus):
    """
    Recibe una lista de líneas que terminan en '\n', y genera otra lista que contiene el mismo
    texto, pero con '\n\n' y '.' como separador.
    También se deshace de todos los '...'
    """
    parsed_corpus = [ '<linejump>' if x == '\n' else x[:-1] for x in contents ]
    parsed_corpus = ' '.join(parsed_corpus).split('<linejump>')
    parsed_corpus = [x.strip() for x in parsed_corpus]
    parsed_corpus = [ x.replace('...','').replace('.', '.<eos>').split('<eos>') for x in parsed_corpus ]
    parsed_corpus = reduce(operator.concat, parsed_corpus)
    parsed_corpus = filter(lambda x: len(x)>0, parsed_corpus)
    parsed_corpus = [x.strip() for x in parsed_corpus]
    return parsed_corpus

In [5]:
corpus_sentences = get_sentences(contents)
corpus_sentences[:10]

['En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero, adarga antigua, rocín flaco y galgo corredor.',
 'Una olla de algo más vaca que carnero, salpicón las más noches, duelos y quebrantos los sábados, lantejas los viernes, algún palomino de añadidura los domingos, consumían las tres partes de su hacienda.',
 'El resto della concluían sayo de velarte, calzas de velludo para las fiestas, con sus pantuflos de lo mesmo, y los días de entresemana se honraba con su vellorí de lo más fino.',
 'Tenía en su casa una ama que pasaba de los cuarenta, y una sobrina que no llegaba a los veinte, y un mozo de campo y plaza, que así ensillaba el rocín como tomaba la podadera.',
 'Frisaba la edad de nuestro hidalgo con los cincuenta años; era de complexión recia, seco de carnes, enjuto de rostro, gran madrugador y amigo de la caza.',
 'Quieren decir que tenía el sobrenombre de Quijada, o Quesada, que en esto hay alguna difer

### Preprocesar oraciones

Pasar todo a minúsculas, y eliminar algunos signos de puntuación (guión largo, comillas, y comillas españolas).

In [6]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

def delete_substrings(string: str, chars: list) -> str:
    final_str = string
    for char in chars:
        final_str = final_str.replace(char, '')
    return final_str

def process_sentences(sentences: list[str], stopwords: list) -> list:
    clean_sentences = [delete_substrings(sent.lower(), stopwords) for sent in sentences]
    # Para que la tokenización funcione necesitamos que los signos de puntuación estén separadas de las palabras.
    # Es decir, algo como "palabra," debe estar como "palabra ,"
    # Haremos esto con el tokenizer de nltk
    processed_sentences = [' '.join(word_tokenize(t)) for t in clean_sentences]
    return processed_sentences

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DELL\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
stopwords = ["''","—","«","»"]
corpus_sentences = process_sentences(corpus_sentences, stopwords)

In [8]:
corpus_sentences[:10]

['en un lugar de la mancha , de cuyo nombre no quiero acordarme , no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero , adarga antigua , rocín flaco y galgo corredor .',
 'una olla de algo más vaca que carnero , salpicón las más noches , duelos y quebrantos los sábados , lantejas los viernes , algún palomino de añadidura los domingos , consumían las tres partes de su hacienda .',
 'el resto della concluían sayo de velarte , calzas de velludo para las fiestas , con sus pantuflos de lo mesmo , y los días de entresemana se honraba con su vellorí de lo más fino .',
 'tenía en su casa una ama que pasaba de los cuarenta , y una sobrina que no llegaba a los veinte , y un mozo de campo y plaza , que así ensillaba el rocín como tomaba la podadera .',
 'frisaba la edad de nuestro hidalgo con los cincuenta años ; era de complexión recia , seco de carnes , enjuto de rostro , gran madrugador y amigo de la caza .',
 'quieren decir que tenía el sobrenombre de quijada , o quesada , qu

### Tokenizar

Vamos a usar subword tokenization, como se muestra en el notebook de la práctica 5.

In [9]:
!pip install subword-nmt

In [10]:
# Entrenamos un tokenizador con BPE usando subword-nmt.
# Para esto necesitamos ingresarle el texto plano del corpus:

def write_plain_text_corpus(raw_text: str, file_name: str) -> None:
    with open(f"{file_name}.txt", "w", encoding="utf-8") as f:
        f.write(raw_text)

plain_corpus = '\n'.join(corpus_sentences)
write_plain_text_corpus(plain_corpus, "plain_processed_quijote")

In [11]:
from collections import Counter

# Primero checamos de qué tamaño es nuestro vocabulario
print("tokens:", len(plain_corpus.split()))
types_in_corpus = Counter(plain_corpus.split())
print("types in corpus:", len(types_in_corpus))

tokens: 425913
types in corpus: 22653


In [12]:
types_in_corpus.most_common(15)

[(',', 39209),
 ('que', 20079),
 ('y', 17688),
 ('de', 17540),
 ('la', 10003),
 ('a', 9565),
 ('en', 7967),
 ('el', 7933),
 ('.', 7715),
 ('no', 6126),
 (';', 4709),
 ('los', 4630),
 ('se', 4549),
 ('con', 4053),
 ('por', 3785)]

In [13]:
!subword-nmt learn-bpe -s 1000 < plain_processed_quijote.txt > quijote_tokenizer.model


100%|##########| 1000/1000 [00:04<00:00, 225.12it/s]


In [14]:
tokenized = !echo "caminando nuestro flamante aventurero" | subword-nmt apply-bpe -c quijote_tokenizer.model
print("tokenized:", list(tokenized)[0].replace('"',''))

tokenized: @@ camin@@ ando nuestro f@@ l@@ am@@ ante aventu@@ r@@ er@@ o@@  


In [15]:
!subword-nmt apply-bpe -c quijote_tokenizer.model < plain_processed_quijote.txt > quijote_tokenized_1k.txt

In [16]:
# Ahora, con esta tokenización, veremos cuántos tokens y tipos hay

with open("quijote_tokenized_1k.txt", "r", encoding="utf-8") as f:
    tokenized_text = f.read()

In [17]:
# Agregamos EOS y BOS
tokenized_text = ['<s> ' + sent + ' </s>' for sent in tokenized_text.split('\n')]
tokenized_text = '\n'.join(tokenized_text).replace('.','')

In [18]:
tokenized_corpus = tokenized_text.split()

In [19]:
print("tokens:", len(tokenized_corpus))
types_in_corpus = Counter(tokenized_corpus)
print("types in corpus:", len(types_in_corpus))

tokens: 649455
types in corpus: 1079


In [20]:
types_in_corpus.most_common(15)

[(',', 39209),
 ('que', 20321),
 ('de', 18185),
 ('y', 18057),
 ('a', 11565),
 ('la', 11521),
 ('<s>', 9248),
 ('</s>', 9248),
 ('en', 9028),
 ('el', 8152),
 ('no', 6995),
 ('los', 5274),
 ('se', 5223),
 (';', 4709),
 ('le', 4348)]

Ahora sí, formamos el corpus con las oraciones tokenizadas

In [21]:
tokenized_corpus_sentences = [ sent.split() for sent in tokenized_text.split('\n') ]

In [22]:
print(tokenized_corpus_sentences[0])

['<s>', 'en', 'un', 'lugar', 'de', 'la', 'mancha', ',', 'de', 'cu@@', 'yo', 'nombre', 'no', 'quiero', 'a@@', 'cor@@', 'dar@@', 'me', ',', 'no', 'ha', 'mucho', 'tiempo', 'que', 'vi@@', 'vía', 'un', 'hi@@', 'd@@', 'algo', 'de', 'los', 'de', 'l@@', 'anza', 'en', 'as@@', 'ti@@', 'll@@', 'ero', ',', 'ad@@', 'ar@@', 'ga', 'an@@', 'ti@@', 'gua', ',', 'ro@@', 'c@@', 'í@@', 'n', 'f@@', 'lac@@', 'o', 'y', 'gal@@', 'go', 'cor@@', 're@@', 'dor', '</s>']


### Diccionario de vocabulario
Nuestro modelo del lenguaje requiere que pasemos nuestras palabras a indices numericos. Utilizaremos enteros para estimar el modelo. 

Crearemos dos diccionarios: 
1. el primero tomara la palabra y lo convertira a indice (Para acceder a las probabilidades del modelo) 
2. El segundo tomará los indices y los convertira de vuelta a palabras (Nos ayudará a recuperar las palabras a partir de los índices del modelo)

In [23]:
vocabulary = list(types_in_corpus.keys())
vocabulary_by_type = { tipo: indice for (tipo, indice) in zip(vocabulary, range(len(vocabulary)))}
vocabulary_by_index = { indice: tipo for (tipo, indice) in zip(vocabulary, range(len(vocabulary)))}

print("index of word 'lugar':  ", vocabulary_by_type['lugar'])
print("word in index 3:        ", vocabulary_by_index[3])

index of word 'lugar':   3
word in index 3:         lugar


### Indexar tokens del corpus

In [24]:
def word_to_index(corpus: list[list[str]], vocab_by_type) -> list[list[int]]:
    """Function that maps each word in a corpus to a unique index"""
    return [ [vocab_by_type[token] for token in sent] for sent in corpus]

In [25]:
indexed_corpus_sentences = word_to_index(tokenized_corpus_sentences, vocabulary_by_type)

In [26]:
print(indexed_corpus_sentences[0])

[0, 1, 2, 3, 4, 5, 6, 7, 4, 8, 9, 10, 11, 12, 13, 14, 15, 16, 7, 11, 17, 18, 19, 20, 21, 22, 2, 23, 24, 25, 4, 26, 4, 27, 28, 1, 29, 30, 31, 32, 7, 33, 34, 35, 36, 30, 37, 7, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 14, 48, 49, 50]


## Dividir en conjuntos de entrenamiento y prueba

In [27]:
from sklearn.model_selection import train_test_split

corpus_train, corpus_test = train_test_split(indexed_corpus_sentences, test_size=0.3)

len(corpus_train) + len(corpus_test) == len(indexed_corpus_sentences)

print("Train len:", len(corpus_train), "test len:", len(corpus_test))

Train len: 6473 test len: 2775


## Estimación del modelo de n-gramas

In [28]:
import numpy as np
from itertools import chain
from collections import defaultdict

def get_n_grams(indexed_sents: list[list[str]], n=2) -> chain:
    """
    Returns one iterable with each of the n-grams of each of the sentences in indexed_sents.
    """
    return chain(*[zip(*[sent[i:] for i in range(n)]) for sent in indexed_sents])

def get_model(sents: list[list[str]], vocabulary: defaultdict, n: int=2, l: float=1.0) -> tuple:
    """
    Params:
        - sents: sentences to train the model with (list of lists of *types indexes*)
        - vocabulary: dictionary where the keys are the types (not indexes), with all the possible types in the corpus
        - n: number of grams
        - l: parameter `l` for Lidstone Smoothing
        
    Returns a tuple (A, Pi), where A is a tensor with all the trasition probabilities in the
    n-grams model, and Pi are the initial probabilities for each type in the vocabulary.
    
    Dimension of A: (N,)*(n-1) + (N,)
    Dimension of Pi: N
    
    Where N is the size of the vocabulary, and n is the n of n-grams.
    """
    BOS_IDX = vocabulary['<s>']
    EOS_IDX = vocabulary['</s>']

    # Get n_grams
    n_grams = get_n_grams(sents, n)

    # Get n_grams frequencies
    freq_n_grams = Counter(n_grams)

    # Get vocabulary length (WITH BOS/EOS)
    N = len(vocabulary)
    # Calculate tensor dimentions for transition probabilities
    dim = (N,)*(n-1) + (N,)

    # Transition tensor
    A = np.zeros(dim)
    # Initial Probabilities
    Pi = np.zeros(N)

    for n_gram, frec in freq_n_grams.items():
        # Fill the tensor with frequencies
        if n_gram[0] != BOS_IDX:
            A[n_gram] = frec
        # Getting initial frequencies
        elif n_gram[0] == BOS_IDX and n_gram[1] != EOS_IDX:
            Pi[n_gram[1]] = frec

    # Calculating probabilities from frequencies
    # We consider the parameter `l` for Lidstone Smoothing
    for h, b in enumerate(A):
        A[h] = ((b+l).T/(b+l).sum(n-2)).T

    # Calculating initial probabilities
    Pi = (Pi+l)/(Pi+l).sum(0)

    # We get our model
    return A, Pi

Estimando un modelo de bigramas con  λ=1

In [29]:
%time
bigram_model = get_model(corpus_train, vocabulary_by_type, n=2, l=1)

CPU times: total: 0 ns
Wall time: 0 ns


In [30]:
A_bigram = bigram_model[0]
print("Tensor dimention", A_bigram.shape)
print("Suma de probabilidades")
print(A_bigram.sum(1))

Tensor dimention (1079, 1079)
Suma de probabilidades
[1. 1. 1. ... 1. 1. 1.]


Ahora de trigramas:

In [31]:
%%time
trigram_model = get_model(corpus_train, vocabulary_by_type, n=3, l=1)

CPU times: total: 24.5 s
Wall time: 33.7 s


In [32]:
A_trigram = trigram_model[0]
print("Tensor dimention", A_trigram.shape)
print("Suma de probabilidades")
print(A_trigram.sum(1))

Tensor dimention (1079, 1079, 1079)
Suma de probabilidades
[[1.         1.         1.         ... 1.         1.         1.        ]
 [0.99525117 1.01029854 1.00672869 ... 0.99525117 0.99525117 0.99525117]
 [0.99886128 1.00883412 0.99977286 ... 0.99886128 0.99886128 0.99886128]
 ...
 [0.99999914 0.99999914 0.99999914 ... 0.99999914 0.99999914 0.99999914]
 [0.99999914 0.99999914 0.99999914 ... 0.99999914 0.99999914 0.99999914]
 [0.99999828 0.99999828 0.99999828 ... 0.99999828 0.99999828 0.99999828]]


## Aplicando el modelo

Para determinar la probabilidad, utilizaremos la función:

$$p(w_1 ... w_k) = \prod_{i=1}^k p(w_i|w_{i-1} ... w_{i-n+1})$$

Dado que las cadenas pueden extenderse y las probabilidades son pequeñas, es posible que la probabilidad se haga tan pequeña que aparezca como un cero. Para evitar esto, utilizaremos probabilidad logarítimicada, dada por:

$$\log p(w_1 ... w_k) = \sum_{i=1}^k \log p(w_i|w_{i-1} ... w_{i-n+1})$$

In [33]:
def get_sent_probability(indexed_sentence: str, model: tuple) -> float:
    """
    Params:
        - indexed_sentence: list of indexes of types
        - model: tuple (A, Pi), where A is a tensor with all the trasition probabilities in the n-grams model, and Pi 
            are the initial probabilities for each type in the vocabulary. This tuple is generated by the get_model function.
    
    Returns the probability of a sentence (list of indexed tokens), using the formula given in the previous cell.
    """
    A, Pi = model
    # Getting the n from n-grams
    n = len(A.shape)
    first_indexed_word = indexed_sentence[0]
    # Getting initial probability
    try:
        probability = np.log(Pi[first_indexed_word])
    except:
        print(f"[WARN] OOV for word as BOS with index={first_indexed_word}")
        probability = 0.0

    # Getting n-grams of the sentence
    n_grams = get_n_grams([indexed_sentence], n)
    for n_gram in n_grams:
        try:
            probability += np.log(A[n_gram])
        except:
            print(f"[WARN] OOV for n_gram={n_gram}")
            probability += 0.0

    return probability

In [34]:
def predict_next_word(indexed_sentence: list[int], model: tuple) -> str:
    """
    Params:
        - indexed_sentence: list of indexes of types
        - model: tuple (A, Pi), where A is a tensor with all the trasition probabilities in the n-grams model, and Pi 
            are the initial probabilities for each type in the vocabulary. This tuple is generated by the get_model function.
            
    Returns the index of the predicted next token (the one with the highest probability) given the previous tokens (the one ins indexed_sentence).
    """
    A, Pi = model
    history = len(A.shape) - 1
    prev_n_gram = tuple(indexed_sentence[-history:])
    probability = get_sent_probability(indexed_sentence, model)
    next_word = np.argmax(probability + np.log(A[prev_n_gram]))
    return next_word

In [35]:
TEST_SENTENCE = corpus_test[0][:-2]
print("Test sentence:", [vocabulary_by_index[i] for i in TEST_SENTENCE])

Test sentence: ['<s>', 'por', 'ver', 'que', 'tiene', 'este', 'caso', 'un', 'no', 'sé', 'qué', 'de', 'so@@', 'm@@', 'b@@', 'ra', 'de', 'a@@', 'ventura', 'de', 'caballería', ',', 'yo', ',', 'por', 'mi', 'parte', ',', 'os', 'o@@', 'i@@', 'r@@', 'é', ',', 'her@@', 'mano', ',', 'de', 'muy', 'buena', 'g@@', 'ana', ',', 'y', 'así', 'lo', 'har@@', 'án', 'todos', 'estos', 'señores', ',', 'por', 'lo', 'mucho', 'que', 'tienen', 'de', 'discre@@', 'tos', 'y', 'de', 'ser', 'ami@@', 'gos', 'de', 'cu@@', 'ri@@', 'o@@', 'sas', 'no@@', 've@@', 'dad@@', 'es', 'que', 'sus@@', 'pen@@', 'd@@', 'an', ',', 'ale@@', 'gr@@', 'en', 'y', 'entre@@', 'ten@@', 'g@@', 'an', 'los', 's@@', 'enti@@', 'dos', ',', 'como', ',', 'sin', 'duda', ',', 'pien@@', 'so', 'que', 'lo', 'ha', 'de', 'hacer', 'vuestro', 'cu@@']


In [36]:
# Prueba bigramas
next_word_index = predict_next_word(TEST_SENTENCE, bigram_model)
vocabulary_by_index[next_word_index]

'ant@@'

In [37]:
# Prueba trigramas
next_word_index = predict_next_word(TEST_SENTENCE, trigram_model)
vocabulary_by_index[next_word_index]

'ento'

# Perplejidad

En el libro de Jurafsky (y en clase) se define la perplejidad como:
$$ perplexity(W) = \sqrt[N]{ \prod_{i=1}^N \frac{1}{p(w_i|w_{1} ... w_{i-1})} } $$

Donde $W$ es una cadena con el conjunto de prueba (por ejemplo, concatenando todas las cadenas del conjunto de prueba), $N$ es la cantidad de tokens en el conjunto de prueba, y $w_i$ es el *i-ésimo* token en $W$.

Para **bigramas** es:
$$ perplexity(W) = \sqrt[N]{ \prod_{i=1}^N \frac{1}{p(w_i|w_{i-1})} } $$

In [38]:
import math

def get_conditional_prob(w_i, indexed_sentence, model, vocabulary):
    """
    Parameters:
        - w_i: index (in our vocabulary dictionary) of some token.
        - indexed_sentence: list of tokens (or rather, their indexes). This list MUST be of size n-1, where n is the number 
            of grmas the model uses (the n in n-grams).
        - model: tuple (A, Pi), where A is a tensor with all the trasition probabilities in the n-grams model, and Pi 
            are the initial probabilities for each type in the vocabulary. This tuple is generated by the get_model function.
        - vocabulary: dictionary where the keys are the types (not indexes), with all the possible types in the corpus
        
    Returns the probability (according to the given model) of token w_i given that the previous tokens were the ones in
    indexed_sentence.
    """
    A, Pi = model
    # Getting the n from n-grams
    n = len(A.shape)
    prev = indexed_sentence
    # Getting n-grams of the sentence
    prev.append(w_i)
    n_grams = tuple(prev)
    if n_grams[0]==vocabulary['</s>'] and n_grams[1]==vocabulary['<s>']:
        return 1
    try:
        probability = A[n_grams]
    except:
        print(f"[WARN] OOV for n_gram={n_grams}")
        probability = 0.0
    
    return probability
    

def get_prerplexity(W, n, model, vocabulary):
    """
    Parameters:
        - W is a list of tokens. It's all the sentences in the test set concatenated.
        - n is the number of grmas the model uses (the n in n-grams)
        - model: tuple (A, Pi), where A is a tensor with all the trasition probabilities in the n-grams model, and Pi 
            are the initial probabilities for each type in the vocabulary. This tuple is generated by the get_model function.
        - vocabulary: dictionary where the keys are the types (not indexes), with all the possible types in the corpus
    
    Returns the perplexity (as defined previously) of the model for test set W.
    """
    N = len(W) 
    probs = [1/get_conditional_prob(W[i], W[i-(n-1): i], model, vocabulary) for i in range(n-1, N)]
    product = math.prod(probs)
    return product**(1/N)

## Perplejidad logarítmica
En las diapositivas, la profesora nos mostró la perplejidad logarítmica:
$$ \log(Perplexity(W)) = -\frac{1}{N} \sum_{i=1}^N \log_2{p(w_i|w_{1} ... w_{i-1})}  $$
De donde:
$$logPerplexity(W) = 2^{-\frac{1}{N} \sum_{i=1}^{N} \log_2 P(w_i | w_1, w_2, \ldots, w_{i-1})}$$

In [39]:
def get_log_perplexity(W, n, model, vocabulary):
    """
    Parameters:
        - W is a list of tokens. It's all the sentences in the test set concatenated.
        - n is the number of grmas the model uses (the n in n-grams)
        - model: tuple (A, Pi), where A is a tensor with all the trasition probabilities in the n-grams model, and Pi 
            are the initial probabilities for each type in the vocabulary. This tuple is generated by the get_model function.
        - vocabulary: dictionary where the keys are the types (not indexes), with all the possible types in the corpus
    
    Returns the log perplexity (as defined previously) of the model for test set W.
    """
    N = len(W)
    log_probs = [math.log(get_conditional_prob(W[i], W[i-(n-1): i], model, vocabulary), 2) for i in range(n-1, N)]
    product = sum(log_probs) * -1/N
    return 2**(product)

In [40]:
flatened_test_set = list(chain(*corpus_test[:500]))
len(flatened_test_set)

35500

In [41]:
perplexity = get_log_perplexity(flatened_test_set, 2, bigram_model, vocabulary_by_type)
print("Perplejidad del modelo de bigramas:", perplexity)

Perplejidad del modelo de bigramas: 119.75132838700031


In [42]:
perplexity_3 = get_log_perplexity(flatened_test_set, 3, trigram_model, vocabulary_by_type)
print("Perplejidad del modelo de bigramas:", perplexity_3)

Perplejidad del modelo de bigramas: 356.80767789670006
